In [1]:
#%pip install langchain
#%pip install "langserve[all]"
#%pip install langchain-cli
#%pip install langchain-openai
#%pip install python-dotenv
#%pip install openai
#%pip install unstructured
#%pip install faiss-cpu
#%pip install chromadb
#%pip install langchainhub
#%pip install pypdf

In [2]:
import openai_setup
import os

In [5]:
from langchain_community.document_loaders import PyPDFLoader

data_path = os.path.join('rag_eval_notebooks','data')

loader = PyPDFLoader('data/azure-machine-learning-azureml-api-2.pdf')
pdf_pages = loader.load_and_split()



In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

page_contents = []


for p in pdf_pages:
    print(p.metadata)
    texts = text_splitter.create_documents([p.page_content])
    for text in texts:
        text.metadata = p.metadata
        page_contents.append(text)

{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 0}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 1}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 2}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 3}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 4}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 5}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 6}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 7}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 8}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 9}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 10}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 11}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 12}
{'source': 'data/azure-machine-learning-azureml-api-2.pdf', 'page': 13}
{'

In [11]:
import os
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings


embeddings_from_model = AzureOpenAIEmbeddings(azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"])
#embeddings = embeddings_model.embed_documents(page_contents)

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_from_model, store, namespace=embeddings_from_model.model
)

In [12]:
list(store.yield_keys())

[]

In [13]:
len(list(store.yield_keys()))

0

In [14]:
from langchain_community.vectorstores import Chroma
chroma_db = Chroma.from_documents(page_contents, AzureOpenAIEmbeddings(), persist_directory="./chroma")

In [15]:
query = "how to integrate aml with spark?"

embedding_vector = AzureOpenAIEmbeddings().embed_query(query)
docs = chroma_db.similarity_search_by_vector(embedding_vector, k=5)
print(docs[0].page_content)

Quickstart: Apache Spark jobs in Azure
Machine Lea rning
Article •05/23/2023
The Azure Machine Learning integration, with Azure S ynapse Analytics, provides easy
access to distributed computing capability - backed by Azure S ynapse - for scaling
Apache Spark jobs on Azure Machine Learning.
In this quickstart guide, you learn how to submit a Spark job using Azure Machine
Learning serverless Spark compute, Azure Data Lake S torage (ADLS) Gen 2 storage
account, and user identity passthrough in a few simple steps.
For more information about Apache Sp ark in Azur e Machine Learning  concepts, see
this resource .
APPLIES T O: 
  Azure CLI ml extension v2 (curr ent)
An Azure subscription; if you don't have an Azure subscription, create a free
account  before you begin.
An Azure Machine Learning workspace. See Create workspace resources .
An Azure Data Lake S torage (ADLS) Gen 2 storage account. See Create an
Azure Data Lake S torage (ADLS) Gen 2 storage account .
Create an Azure Machine Learn